# Monte Carlo Simulations
This notebook describes the basic method used to construct the Monte Carlo Simulation presented in chapter 6 of Applied Math for Security. Much of the code is described in detail throughout the chapter so the descriptions here are meant to supplement the text.

The user_to_series function has been taken from the code presented in chapter 5 but essentially it loads the nested JSON user object into a Pandas Series which can be appended to the main DataFrame object.
The unique function is a (poor) implementation to find all the unique items in a list. This could be replaced by a better implementation from a library like numpy.unique

In [1]:
from random import choice
import graph_funcs as ext
import networkx as nx
import pandas as pd
import networkx as nx
import json

def user_to_series(dict_obj):
    """Convert a nested JSON user to a flat pandas series"""
    renamed = {}
    for k in dict_obj.keys():
        nk = "user_%s" % k
        v = dict_obj[k]
        renamed[nk] = v
    ret = pd.Series(renamed)
    return ret

def unique(bag):
    ret = []
    for i in bag:
        #print(i)
        if i not in ret:
            ret.append(i)
    return ret

We load the posts data using the same method presented in chapter 5.

In [2]:
series_data = [] # 1 JSON object per post object
with open("fake_posts.json") as data:
    text = data.read().strip()
    rows = text.split("\n")  # JSON objects stored as list of strings
for row in rows:
    obj = json.loads(row) # Converted row string to JSON object
    series_data.append(obj) # Add to JSON list
post_df = pd.DataFrame(series_data) # 1 row per JSON obj
post_df = pd.concat([post_df, post_df['user'].apply(user_to_series)], axis=1)
# Now the data is flattened. We remove the field containing the JSON object
post_df.drop("user", axis=1, inplace=True)
post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28034 entries, 0 to 28033
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       28034 non-null  int64  
 1   id_str                   28034 non-null  object 
 2   created_at               28034 non-null  object 
 3   text                     28034 non-null  object 
 4   source                   28034 non-null  object 
 5   retweet_count            28034 non-null  int64  
 6   liked_count              28034 non-null  int64  
 7   in_reply_to_user_id      10302 non-null  object 
 8   in_reply_to_tweet_id     10302 non-null  float64
 9   in_reply_to_user_id_str  10302 non-null  object 
 10  in_reply_to_screen_name  10302 non-null  object 
 11  user_id                  28034 non-null  object 
 12  user_screen_name         28034 non-null  object 
 13  user_location            28034 non-null  object 
 14  user_description      

The code from listing 6-2 is a deterministic simulation where the a message is passed from user to user. The state machine is deterministic although it still uses random choice, because if we choose an input like send the action succeeds 100% of the time. Determinism refers to the actions themselves and not to the result as a whole (which will be stochastic). in the real world, calls may fail. emails may bounce. messages may not successfully be delivered. You may want to extend this code to add cases where a state transition may fail with some probability.

In [3]:
### From Listing 6-2 ###
def run_sim_1(posts):
    G = nx.DiGraph()
    ### From listing 6-1 ###
    XI = ["send", None]
    k=10
    n=10
    posts = posts[posts["in_reply_to_screen_name"].notnull()]
    for idx in posts.index:
        row = posts.loc[idx]
        G.add_edge(row["in_reply_to_screen_name"], row["user_screen_name"], capacity=len(row["text"]))
    out_deg = G.out_degree()
    valkey_sorted = sorted(out_deg, key=lambda x: (x[1], x[0]))
    S0 = valkey_sorted[-1][0]
    ###################
    R = []
    for i in range(k):
        uq = S0
        Tn = []
        for j in range(n):
            if choice(XI) is not None:
                gamma_uq = list(nx.neighbors(G, uq))
                if len(gamma_uq) > 1:
                    vq = choice(gamma_uq)
                    Tn.append((uq, vq))
                    uq = vq
                elif len(gamma_uq) == 1:
                    vq = gamma_uq[0]
                    Tn.append((uq, vq))
                    uq = vq
                else:
                    conc = "Message terminated at node %s in %d steps"
                    print( conc % (uq, len(Tn)))
                    break
    
        R.append((uq, Tn))
    tot = 0
    for end, path in R:
        conc = "Message reached node %s in %d steps"
        print( conc % (uq, len(path)))
        uniq = unique(path)
        tot = len(uniq) - 1
    print(S0, (tot / len(R)) / (len(G.nodes.keys()) - 1))

run_sim_1(post_df)

Message terminated at node bethanyturner in 2 steps
Message terminated at node fbrown in 3 steps
Message terminated at node fbrown in 5 steps
Message terminated at node bethanyturner in 2 steps
Message terminated at node fbrown in 3 steps
Message terminated at node bethanyturner in 3 steps
Message terminated at node bethanyturner in 2 steps
Message reached node bethanyturner in 3 steps
Message reached node bethanyturner in 4 steps
Message reached node bethanyturner in 2 steps
Message reached node bethanyturner in 6 steps
Message reached node bethanyturner in 3 steps
Message reached node bethanyturner in 5 steps
Message reached node bethanyturner in 2 steps
Message reached node bethanyturner in 3 steps
Message reached node bethanyturner in 3 steps
Message reached node bethanyturner in 2 steps
dannyhoover 0.0012048192771084338


The next cell wraps the code from listing 6-3 in a convenience function definition which rerun the simulation for each term you pass in the term_list input parameter. The simulation uses the hub part of the HITS algorithm to determine which node would likely create a post based on the term. 

In [4]:
### From listing 6-3 ###
def run_sim_2(term_list, post_df):
    for term in term_list:
        R = []
        hG = ext.term_subgraph(term, post_df)
        hub_scores, auth_scores = nx.hits(hG, max_iter=1000, tol=0.01)
        hub_max = max(hub_scores.values())
        S0_i = list(hub_scores.values()).index(hub_max)
        S0 = list(hub_scores.keys())[S0_i]

        for i in range(k):
            uq = S0
            Tn = []
            for j in range(n):
                send_msg = ext.hub_send(hub_scores[uq])
                if send_msg:
                    vq = ext.scored_neighbor_select(hG, uq, auth_scores)
                    if vq is None:
                        conc = "Message terminated at node %s in %d steps"
                        print( conc % (uq, len(Tn)))
                        break
                    else:
                        Tn.append((uq, vq))
                        uq = vq
            R.append((uq, Tn))
        ended_at = {}
        for end, path in R:
            if end in ended_at.keys():
                ended_at[end] += 1
            else:
                ended_at[end] = 1
        return (S0, ended_at)


The next cell runs the run_sim_2 function ten times and aggregate the result

In [5]:
### From Listing 6-4 ###
all_runs = {}
started_at = ""
k=10
n=10
for run_i in range(10):
    started_at, results = run_sim_2(["environment"], post_df)
    for ks in results:
        if ks in all_runs.keys():
            all_runs[ks] += results[ks]
        else:
            all_runs[ks] = results[ks]
for node in all_runs.keys():
    if node != started_at:
        print("%s influenced %s an average of %.2f times" % (
            started_at, node, all_runs[node]/10
        ))

gutierrezjamie influenced shannon42 an average of 0.90 times
gutierrezjamie influenced hartmanmatthew an average of 2.70 times
gutierrezjamie influenced garciajames an average of 1.50 times
gutierrezjamie influenced iwatkins an average of 2.50 times
gutierrezjamie influenced daniel99 an average of 0.60 times
gutierrezjamie influenced grosslinda an average of 1.00 times
